In [1]:
import pandas as pd
import numpy as np
import random
from sympy import isprime, nextprime
import os
import pickle
import random
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df1 = pd.read_csv('6_1_CC_Encrypted_Data_Part.csv')
df1

,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol
0,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
1,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
2,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
3,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
4,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
...,...,...,...,...,...
1817,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
1818,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
1819,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...
1820,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...


In [7]:
df2 = pd.read_csv('6_2_CC_Encrypted_Data_Part.csv')
df2

,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak
0,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
1,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
2,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
3,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
4,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
...,...,...,...,...,...
1817,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
1818,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
1819,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...
1820,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...


In [8]:
df3 = pd.read_csv('5_3_CC_Encrypted_Data_Part.csv')
df3

,ST_Slope,HeartDisease
0,(102350512016187766238753594805484016713153736...,0
1,(102350512016187766238753594805484016713153736...,1
2,(102350512016187766238753594805484016713153736...,0
3,(102350512016187766238753594805484016713153736...,1
4,(102350512016187766238753594805484016713153736...,0
...,...,...
1817,(102350512016187766238753594805484016713153736...,0
1818,(102350512016187766238753594805484016713153736...,0
1819,(102350512016187766238753594805484016713153736...,0
1820,(102350512016187766238753594805484016713153736...,0


In [9]:
df_combined = pd.concat([df1, df2, df3], axis='columns')
df_combined

,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,0
1,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,1
2,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,0
3,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,1
4,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1817,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...,(102350512016187766238753594805484016713153736...,0
1818,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582

In [10]:
df_combined.to_csv('7_CC_Encrypted_Data.csv', index=False)

In [11]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

mismatch_summary = {}
random_indices = random.sample(range(len(df_combined)), 500)
for i in range(len(random_indices)): print(random_indices[i] + 2)

974
1009
758
241
708
1429
954
28
539
1549
1749
1242
1167
327
1427
1458
1751
1012
1768
1784
69
529
794
559
423
593
772
1645
348
732
118
5
1263
1797
1662
209
391
1779
315
1637
8
230
102
131
570
249
1531
1213
1428
976
1348
352
453
1649
217
6
275
562
1051
1385
1095
604
633
526
1316
213
1612
1363
1820
1550
268
575
1465
1636
372
1311
1696
166
1091
1702
1251
868
1041
158
1312
1373
795
1534
1601
1106
1105
40
427
1470
1562
1781
78
48
1028
865
1139
727
673
114
1403
773
1093
104
60
1561
1169
328
13
1487
1570
598
840
1339
1256
1030
899
571
1559
845
1341
720
1277
236
183
931
1022
1675
1368
662
779
597
1023
1740
1500
600
813
966
682
459
46
18
614
256
232
301
409
1613
810
1681
1437
193
353
749
617
550
311
857
496
620
951
591
1485
139
137
465
200
334
424
1468
269
350
1551
671
1102
1293
1668
197
151
1634
1011
376
1421
854
1707
991
72
134
1013
1280
1593
757
1584
203
670
1332
860
188
226
537
1752
1711
1078
1314
329
658
738
823
663
1451
431
1261
805
1665
1449
1302
515
1679
1691
705
446
910
437
1521
781
56

In [12]:
df = pd.read_csv('7_CC_Encrypted_Data.csv')
df_org = pd.read_csv('1_CC_Structured_Data.csv')
private_key = load_key("3_CC_Private_Key.pkl")

In [13]:
for k in range(10):
    x, p, g = private_key['x'], private_key['p'], private_key['g']
    
    messages = df.iloc[random_indices, k].values
    original_column = df_org.iloc[random_indices, k].values  

    decrypted_column = []
    mismatches = [] 

    for i, (message, original_value) in zip(random_indices, zip(messages, original_column)):
        if message is not None and message == message:
            
            c1, c2 = map(int, message.strip("()").split(","))
            cur = power(c1, x, p)
            if cur == 0:
                decrypted_column.append(None)
                continue
                
            temp = mod_inv(cur, p)
            decrypted_message = (c2 * temp) % p
   
            temp_decrypted_message = int_to_string(decrypted_message)         
                
            decrypted_column.append(temp_decrypted_message)
                
            if temp_decrypted_message != original_value:
                mismatches.append(i + 2)
                print(f"dec: {temp_decrypted_message}, org: {original_value}")
        else:
            decrypted_column.append(None)

    mismatch_summary[column_names[k]] = mismatches
    print(f"Mismatches for {column_names[k]}: {mismatches}")

print("\nMismatch Summary:")
for column, mismatches in mismatch_summary.items():
    print(f"{column}: {mismatches}")


Mismatches for Age: []
Mismatches for Sex: []
Mismatches for ChestPainType: []
Mismatches for RestingBloodPressure: []
Mismatches for Cholesterol: []
Mismatches for FastingBloodSugar: []
Mismatches for RestingECG: []
Mismatches for MaximumHeartRate: []
Mismatches for ExerciseAngina: []
Mismatches for Oldpeak: []

Mismatch Summary:
Age: []
Sex: []
ChestPainType: []
RestingBloodPressure: []
Cholesterol: []
FastingBloodSugar: []
RestingECG: []
MaximumHeartRate: []
ExerciseAngina: []
Oldpeak: []
